# Model Result Visualization

In [1]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import Policy
from problem_formulation import get_model_for_problem_formulation
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# Cell 2: Load model for PF6
pf = 6
model, steps = get_model_for_problem_formulation(pf)
uncertainties = model.uncertainties
levers = model.levers

In [3]:
# Cell 3: Create widgets for uncertainties and levers
unc_widgets = {}
for unc in uncertainties:
    if hasattr(unc, 'categories'):
        unc_widgets[unc.name] = widgets.Dropdown(
            options=[cat.value for cat in unc.categories],
            description=unc.name
        )
    elif unc.name.endswith("_pfail"):
        unc_widgets[unc.name] = widgets.FloatSlider(
            min=unc.lower_bound, max=unc.upper_bound, step=(unc.upper_bound-unc.lower_bound)/100, description=unc.name
        )
    else:
        unc_widgets[unc.name] = widgets.IntSlider(
            min=unc.lower_bound, max=unc.upper_bound, step=(unc.upper_bound-unc.lower_bound)/100, description=unc.name
        )

lev_widgets = {}
for lev in levers:
    lev_widgets[lev.name] = widgets.IntSlider(
        min=lev.lower_bound, max=lev.upper_bound, step=(lev.upper_bound-lev.lower_bound)/100, description=lev.name
    )

In [4]:
# Cell 4: Display widgets and run model
from ema_workbench import perform_experiments, Scenario, MultiprocessingEvaluator
from ema_workbench import ema_logging

def run_and_plot(**kwargs):
    #print("run_and_plot called")
    # Split kwargs into uncertainties and levers
    unc_dict = {k: kwargs[k] for k in unc_widgets}
    lev_dict = {k: kwargs[k] for k in lev_widgets}
    policy = Policy('custom', **lev_dict)
    scenario = Scenario('custom', **unc_dict)
    print("Scenario and policy created, proceeding to run model...")
    # Run model for a single scenario using MultiprocessingEvaluator
    ema_logging.log_to_stderr(ema_logging.INFO)
    with MultiprocessingEvaluator(model) as evaluator:
        experiments, outcomes = evaluator.perform_experiments(scenarios=[scenario], policies=[policy])
    print("Model run completed")

    # Plot all disaggregated outcomes
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    # Define the disaggregated outcome names
    disaggregated_outcome_names = [
        "Expected Annual Damage",
        "Expected Number of Deaths",
        "Total Costs",
        "Dike Investment Costs"
    ]
    # For each outcome, plot all dikes (assume outcomes are named like "Expected Annual Damage Dike 0", ...)
    for i, outcome_base in enumerate(disaggregated_outcome_names):
        ax = axes.flatten()[i]
        dike_outcomes = [name for name in outcomes if outcome_base in name]
        values = [outcomes[name][0] for name in dike_outcomes]
        labels = [name.split('_')[0] for name in dike_outcomes]
        bars = ax.bar(range(len(dike_outcomes)), values, tick_label=labels)
        ax.set_title(outcome_base)
        ax.tick_params(axis='x', rotation=45)
        # Show values above each bar
        for bar, value in zip(bars, values):
            ax.annotate(f'{value:.2f}', xy=(bar.get_x() + bar.get_width() / 2, value),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)
    plt.tight_layout()
    plt.show()

In [ ]:
# Cell 5: Interactive UI

# Create submit and reset buttons
submit_btn = widgets.Button(description="Submit", button_style='success')
reset_btn = widgets.Button(description="Reset", button_style='warning')

# Output widget for plot
plot_out = widgets.Output()

def on_submit_clicked(b):
    with plot_out:
        clear_output(wait=True)
        # Gather current values from widgets
        unc_values = {k: w.value for k, w in unc_widgets.items()}
        lev_values = {k: w.value for k, w in lev_widgets.items()}
        run_and_plot(**unc_values, **lev_values)

def on_reset_clicked(b):
    # Reset all widgets to their initial values
    for k, w in unc_widgets.items():
        if isinstance(w, widgets.IntSlider):
            w.value = w.min if w.min is not None else 0
        elif isinstance(w, widgets.Dropdown):
            w.value = w.options[0]
    for k, w in lev_widgets.items():
        if isinstance(w, widgets.IntSlider):
            w.value = w.min if w.min is not None else 0
        elif isinstance(w, widgets.Dropdown):
            w.value = w.options[0]

submit_btn.on_click(on_submit_clicked)
reset_btn.on_click(on_reset_clicked)

# Set widget layout and style to show full parameter names
style = {'description_width': '250px'}

for w in unc_widgets.values():
    w.layout = widgets.Layout(width='400px')
    if hasattr(w, 'style'):
        w.style = style

for w in lev_widgets.values():
    w.layout = widgets.Layout(width='400px')
    if hasattr(w, 'style'):
        w.style = style

# Create VBox for uncertainties if not already defined
unc_box = widgets.VBox(list(unc_widgets.values()))
lev_box = widgets.VBox(list(lev_widgets.values()))

# Layout: buttons above sliders
button_box = widgets.HBox([submit_btn, reset_btn])
ui = widgets.VBox([button_box, widgets.HBox([unc_box, lev_box])])

display(plot_out, ui)


Output()